In [1]:
import pandas as pd
import numpy as np

## Initialization

Create the base network as a matrix of costs, where a cost of NaN represents a non-adjacent node. The cost to the same node (diagonal of the matrix) is also listed as NaN for programming purposes. Typically, the diagonal should be listed as a 0, but this creates problems in the scripts for Djikstra's Algorithm and the Bellman-Ford Algorithm.

In [2]:
# Manually initialized adjacency matrix
network = pd.DataFrame(
    [[0,5,1,np.nan,np.nan,np.nan],
     [np.nan,0,4,6,np.nan,np.nan],
     [np.nan,np.nan,0,7,2,np.nan],
     [np.nan,np.nan,np.nan,0,np.nan,2],
     [np.nan,1,np.nan,3,0,8],
     [np.nan,np.nan,np.nan,np.nan,np.nan,0]])

network.index = ['Node 1', 'Node 2', 'Node 3', 'Node 4', 'Node 5', 'Node 6']
network.columns = network.index

display(network)

,Node 1,Node 2,Node 3,Node 4,Node 5,Node 6
Node 1,0.0,5.0,1.0,NaN,NaN,NaN
Node 2,NaN,0.0,4.0,6.0,NaN,NaN
Node 3,NaN,NaN,0.0,7.0,2.0,NaN
Node 4,NaN,NaN,NaN,0.0,NaN,2.0
Node 5,NaN,1.0,NaN,3.0,0.0,8.0
Node 6,NaN,NaN,NaN,NaN,NaN,0.0


## Djikstra's Algorithm

In [ ]:
# Initialize the temporary labels, permanent labels, and optimal policy vectors
permanent = pd.Series([0,np.nan,np.nan,np.nan,np.nan,np.nan])
temp = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

temp.index = permanent.index = network.index

In [ ]:
previousPermanentNodeIndex = 0
iteration = 1

while permanent.isnull().values.any():
    
    # Print iteration number
    print(f'Iteration {iteration} permanent costs')

    # Create copy of new possible node costs and set the previous
    # permanent node cost to NaN to avoid an infinite loop caused by the 0 value attached
    # to the same node
    newPossibleNodeCosts = network.iloc[previousPermanentNodeIndex].copy()
    newPossibleNodeCosts.iloc[previousPermanentNodeIndex] = np.nan

    # Calculate the new possible node costs using the previous permanent node
    newNodeCosts = newPossibleNodeCosts + permanent.iloc[previousPermanentNodeIndex]

    # Find min between new possible node costs and temporary costs, call it calcCosts
    calcCosts = np.fmin(newNodeCosts, temp)

    # Find min of the calcCosts and the associated node, make the node
    # and the cost permanent
    newMin = np.nanmin(calcCosts)
    newMinIndex = np.argmin(calcCosts)

    # Update the permanent costs vector
    permanent.iloc[newMinIndex] = newMin
    print(permanent)

    # Update the temporary costs vector
    temp = np.fmin(calcCosts, temp)
    # Make sure that any permanent nodes aren't included
    temp.loc[permanent.notna()] = np.nan


    # Update the previousPermanentNodeIndex
    previousPermanentNodeIndex = newMinIndex

    # Update iteration for display purposes
    iteration += 1

### Finding optimal path policy

In [ ]:
def findPredecessorsDjikstra(targetNode, network, optimalCosts):
    # Iterate through the nodes that are adjacent to the target node unless it's the first node
    if targetNode == 'Node 1':
        return(('Node 1',))

    # Find nodes that are adjacent to the target node and set the cost of same node to NaN so
    # that it isn't considered
    connectingCosts = network.loc[:,targetNode].copy()
    connectingCosts.loc[targetNode] = np.nan

    # Iterate through adjacent nodes
    for adjacentNode in connectingCosts.index:
        
        # If the node is not adjacent, ignore it
        if pd.isna(connectingCosts[adjacentNode]):
            continue
        
        # Check if the cost to get to an adjacent node + the cost to get to target node is equal or
        # less than the optimal cost--if so, the adjacent node is a predecessor
        if optimalCosts[adjacentNode] + connectingCosts[adjacentNode] == optimalCosts[targetNode]:
            return findPredecessorsDjikstra(adjacentNode, network, optimalCosts) + (targetNode,)    

In [ ]:
# Save a Series with the optimal policy
policyDjikstra = pd.Series(index=network.columns, dtype=object)

for node in network.columns:
    policyDjikstra[node] = findPredecessorsDjikstra(node, network, permanent)

display(policyDjikstra)

## Bellman-Ford Algorithm

In [ ]:
# # Initialize costs and policy vectors
# nodeCosts = pd.Series([0, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index)
# previousCosts = pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index)

# policyBellman = pd.Series(index=network.index, dtype=object)
# previousPolicyBellman = pd.Series(index=network.index, dtype=object)

# # Iterate the amount of nodes in the network
# # While Bellman-Ford only needs V - 1 iterations, we run V to check for negative cycles
# for i in range(len(nodeCosts)):
    
#     # Check for negative cycle, detected if i reaches the Vth iteration
#     if i == len(nodeCosts):
#         print("There was a negative cycle in the network")
#     else:
#         print(f"Iteration {i+1}")

#     # For each iteration, iterate through all nodes to see if a smaller
#     # cost can be found to a connecting/adjacent node
#     for nodeIndex in nodeCosts.index:
#         checkCosts = network.loc[nodeIndex].copy() + nodeCosts[nodeIndex]
#         nodeCosts = np.fmin(checkCosts, nodeCosts)
#     print(nodeCosts)

#     # Check if nodeCosts was updated at all or not
#     # If not, the nodeCosts have converged and we can break the loop
#     if pd.Series.equals(nodeCosts, previousCosts):
#         print("The node costs have converged")
#         break
#     # Otherwise, we must update the policy vector
#     else:
#         # For each node, check if we need to update the policy vector
#         for nodeIndex in nodeCosts.index:
#             if nodeCosts[nodeIndex] != previousCosts[nodeIndex]:
#                 # If we haven't updated the policy yet, initialize the tuple
#                 if policyBellman[nodeIndex] is np.nan:
#                     policyBellman[nodeIndex] = (policyBellman[nodeIndex],)
#                 # Update policyBellman otherwise
#                 else:
#                     policyBellman[nodeIndex] = policyBellman[nodeIndex] + (nodeIndex,)
#     print(policyBellman)

#     # Update the previous costs and policy vector for the next iteration
#     previousCosts = nodeCosts
#     previousPolicyBellman = policyBellman

# display(policyBellman)

In [3]:
# Initialize costs, policy vectors, and predecessors
nodeCosts = pd.Series([0, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index)
previousCosts = pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index)
predecessors = pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan], index=network.index)

# Iterate the amount of nodes in the network
# While Bellman-Ford only needs V - 1 iterations, we run V to check for negative cycles
for i in range(len(nodeCosts)):
    
    # Check for negative cycle, detected if i reaches the Vth iteration
    if i == len(nodeCosts):
        print("There was a negative cycle in the network")
    else:
        print(f"Iteration {i+1}")

    # For each iteration, iterate through all starting nodes to see if a smaller
    # cost can be found to a connecting/adjacent end node
    for startNodeIndex in nodeCosts.index:
        for endNodeIndex in nodeCosts.index:
            if startNodeIndex != endNodeIndex:
                checkCost = network.loc[startNodeIndex][endNodeIndex].copy() + nodeCosts[startNodeIndex]
                # If a smaller cost is found, update the cost and the predecessor
                if nodeCosts[endNodeIndex] == np.nan or checkCost < nodeCosts[endNodeIndex]:
                    nodeCosts[endNodeIndex] = checkCost
                    predecessors[endNodeIndex] = startNodeIndex
    print(nodeCosts)

    # Check if nodeCosts was updated at all or not
    # If not, the nodeCosts have converged and we can break the loop
    if pd.Series.equals(nodeCosts, previousCosts):
        print("The node costs have converged")
        break

    # Update the previous costs and policy vector for the next iteration
    previousCosts = nodeCosts

print("Predecessors:", predecessors)

Iteration 1
Node 1    0.0
Node 2    NaN
Node 3    NaN
Node 4    NaN
Node 5    NaN
Node 6    NaN
dtype: float64
Iteration 2
Node 1    0.0
Node 2    NaN
Node 3    NaN
Node 4    NaN
Node 5    NaN
Node 6    NaN
dtype: float64
The node costs have converged
Predecessors: Node 1   NaN
Node 2   NaN
Node 3   NaN
Node 4   NaN
Node 5   NaN
Node 6   NaN
dtype: float64


## Floyd Algorithm

Initialize distance matrix using the original network matrix

In [ ]:
distances = network

display(distances)

In [ ]:
# Iterate through each possible intermediate node
for intermediateNode in distances.index:
    # Iterate through each possible start node
    for startNode in distances.index:
        # Iterate through each possible end node
        for endNode in distances.columns:
            # Update the distance if a shorter path is found
            distances.loc[startNode,endNode] = np.nanmin([distances.loc[startNode,endNode], distances.loc[startNode,intermediateNode] + distances.loc[intermediateNode,endNode]])

display(distances)
